# Crypto_Scraper

## Import 

In [1]:
import praw
import pandas as pd
from datetime import datetime
import os
import psaw as ps
from dotenv import load_dotenv
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

In [2]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [3]:
# Verbuindung zur Pushshift-API
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [4]:
# Test der Verbindung
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: Monthly Optimists Discussion - February 2025, Score: 10
Title: Daily Crypto Discussion - February 2, 2025 (GMT+0), Score: 22
Title: Justin Bieber Was Spotted Looking Like He Held Ethereum Since The Merge, Score: 175
Title: The Most Bullish Bullrun Ever, Score: 128
Title: The Maga backlash against Trump’s crypto grab: ‘This is bad, and looks bad’, Score: 2872


## Liste der Cryptos

In [5]:
# Die Crypto-Liste
cryptos = [
    ("Cardano", "ADA"),
    ("Solana", "SOL"),
    ("Binance Coin", "BNB"),
    ("XRP", "XRP"),
    ("Polkadot", "DOT"),
    ("Dogecoin", "DOGE")
]


In [6]:
# Die Subreddit Liste
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance"        # Diskussionen über die Binance-Plattform  
]

## Scrape

In [7]:
# Funktion, um Posts und Kommentare zu scrapen
def scrape_reddit(start_timestamp, end_timestamp, mode="initial"):
    posts = []
    comments = []
    post_ids = set()  # Set zum Speichern der Post-IDs

    for crypto_name, crypto_symbol in cryptos:
        for subreddit_name in subreddits:
            subreddit = reddit.subreddit(subreddit_name)
            print(f"Suche Posts zu {crypto_name} ({crypto_symbol}) in r/{subreddit_name}...")

            # Suche nach Namen und Kürzel
            for search_term in [crypto_name, crypto_symbol]:
                for post in subreddit.search(query=search_term, sort="new", limit=None):
                    if start_timestamp <= post.created_utc <= end_timestamp and post.id not in post_ids:
                        post_ids.add(post.id)
                        posts.append({
                            'crypto': crypto_name,
                            'search_term': search_term,
                            'subreddit': subreddit_name,
                            'post_id': post.id,
                            'title': post.title,
                            'author': str(post.author),
                            'created_utc': datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                            'score': post.score,
                            'num_comments': post.num_comments,
                            'selftext': post.selftext
                        })
                        print(f"Gefunden in r/{subreddit_name}: {post.title} (Suchbegriff: {search_term})")

                        # Kommentare sammeln
                        post.comments.replace_more(limit=0)
                        for comment in post.comments.list():
                            comments.append({
                                'post_id': post.id,
                                'comment_id': comment.id,
                                'author': str(comment.author),
                                'created_utc': datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                'score': comment.score,
                                'body': comment.body
                            })

    # In DataFrames umwandeln
    df_posts = pd.DataFrame(posts)
    df_comments = pd.DataFrame(comments)

    # DataFrames zurückgeben
    print(f"Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gefunden.")
    return df_posts, df_comments

# **Wöchentlicher Scrape**
now = int(datetime.now().timestamp())  # Aktueller Zeitstempel
last_week = now - 7 * 24 * 60 * 60  # 7 Tage zurück
print("Starte den wöchentlichen Scrape...")
df_posts_weekly, df_comments_weekly = scrape_reddit(last_week, now, mode="weekly")

# Beispiel: Lokale Weiterverarbeitung
print("Daten können jetzt in der Pipeline bereinigt werden...")

Starte den wöchentlichen Scrape...
Suche Posts zu Cardano (ADA) in r/CryptoCurrency...
Gefunden in r/CryptoCurrency: What's the reason crypto community members tend to worship certain indivudals? (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Cardano Plomin Hard Fork Goes Live Bringing Full Decentralized Governance to ADA (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Cardano’s Plomin Hard Fork Goes Live, Ushering in On-Chain Governance (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Cardano Set to Achieve Full Decentralized Governance with Wednesday’s Plomin Hard Fork (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Understanding the "inverse r/cryptocurrency" trading strategy. Is this sub consistently wrong? How it works, how to exploit it, and the underlying reason driving it: (Suchbegriff: Cardano)
Gefunden in r/CryptoCurrency: Tuttle Capital Seeks SEC Approval for First-Ever Chainlink, Cardano, and Polkadot ETFs (Suchbegriff: Cardano)
Gefunden in r/CryptoCurre

## Datenbereinigung 

Fubktion zur Bereinigung der Daten 

In [8]:
def clean_data(df_posts, df_comments):
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])
    
    # 4. Filterung nach Qualität (z. B. Spam entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen
    
    return df_posts, df_comments


Funktion zur bereinigung auf die Daten anwenden

In [9]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts_weekly, df_comments_weekly)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

Bereinigte Posts: 60
Bereinigte Kommentare: 4120


## Model fuer das Sentiment

In [10]:
df_comments_clean

,post_id,comment_id,author,created_utc,score,body
0,1if3ko4,macv3h8,Busy-Chemistry7747,2025-02-01 10:39:21,53,Room temperature iq
1,1if3ko4,macukdp,Maleficent_Sound_919,2025-02-01 10:33:59,22,Cult like worshipping
2,1if3ko4,macxctn,inShambles3749,2025-02-01 11:01:49,7,Because humans are dumb fucks and tend to go t...
3,1if3ko4,macvmq8,liquid_at,2025-02-01 10:44:46,7,chicken and egg problem.\n\nThe psychological ...
4,1if3ko4,macwj7f,Bear-Bull-Pig,2025-02-01 10:53:46,6,I think most crypto members are here for the m...
...,...,...,...,...,...,...
4661,1ifhr6g,magor0q,kilo6ronen,2025-02-01 23:48:59,1,This sentiment happens literally every single ...
4662,1ifhr6g,magajon,Emergency_Egg1281,2025-02-01 22:32:00,1,"I feel you , I had to vent, I am pissed off no..."
4663,1ifhr6g,magsrkp,Emergency_Egg1281,2025-02-02 00:10:46,1,"trump at 19 ,,,,bought it !"
4664,1icsxbw,m9v8qtn,Mrqs2,2025-01-29 18:41:02,3,Honestly at this point I don’t even know what’...


In [ ]:
# 🔹 FinBERT-Modell & Tokenizer laden
MODEL_NAME = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

df = df_comments_clean

# 🔹 Sicherstellen, dass die Spalte "body" existiert
if "body" not in df.columns:
    raise ValueError("❌ Fehler: Die CSV-Datei enthält keine 'body'-Spalte mit Kommentaren!")

# 🔹 Funktion zur Sentiment-Analyse
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0  # Leere Kommentare sind neutral

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = F.softmax(outputs.logits, dim=1)[0]
    labels = ["negative", "neutral", "positive"]
    sentiment = labels[torch.argmax(scores).item()]
    confidence = scores.max().item()

    return sentiment, confidence

# 🔹 Sentiment für alle Kommentare berechnen
df["sentiment"], df["sentiment_confidence"] = zip(*df["body"].map(analyze_sentiment))


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

c:\Users\hp\anaconda3\envs\dennis\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ Sentiment-Analyse abgeschlossen! Neue Datei gespeichert: reddit_comments_with_sentiment.csv


In [ ]:
df.sentiment_confidence.sort_values(ascending=False)

## Google Drive Export

Pfad

In [14]:
# Definiere den Speicherpfad zu deinem lokalen Google Drive
DRIVE_PATH = "G:/Meine Ablage/"


Export Funktion

In [15]:
def append_to_csv(df_new, filename):
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls die Datei bereits existiert, lese sie ein und hänge die neuen Daten an
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep=";", encoding="utf-8-sig", on_bad_lines="skip")
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new  # Falls die Datei nicht existiert, erstelle sie neu

        # CSV speichern mit besserer Formatierung für Excel
        df_combined.to_csv(
            file_path,
            index=False,
            sep=";",  # Semikolon als Trennzeichen für bessere Excel-Kompatibilität
            encoding="utf-8-sig",  # Richtige Zeichenkodierung für Excel
            lineterminator="\n"  # Zeilenumbrüche korrekt speichern
        )
        print(f"Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments):
    try:
        # Posts an bestehende Datei anhängen
        append_to_csv(df_posts, "reddit_posts.csv")
        # Kommentare an bestehende Datei anhängen
        append_to_csv(df_comments, "reddit_comments.csv")

    except Exception as e:
        print(f"Fehler beim Export: {e}")


Export Funktion anwenden

In [16]:
# Export der bereinigten Daten
export_to_drive(df_posts_clean, df_comments_clean)

Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_posts.csv
Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit_comments.csv
